In [1]:
import pandas as pd
import numpy as np
import openai
from openai.types.beta.threads.message_create_params import (
    Attachment,
    AttachmentToolFileSearch,
)

import picologging as logging

from pydantic import BaseModel, Field

import pathlib
import datetime
import asyncio
from asyncio import Semaphore
import json

import httpx

from typing import Optional, List, Union, Literal, Tuple

In [2]:
with open("../tokens/openai_token.txt") as f:
    openai_token = f.read()

In [3]:
now = datetime.datetime.now()
now_str = now.strftime("%Y%m%d_%H%M%S")
pathlib.Path(f"../notebooks_logging/06_ask_q4pages").mkdir(parents=True, exist_ok=True)
# pathlib.Path(f"../notebooks_logging/05_cut_pdfs/{now_str}").mkdir(parents=True, exist_ok=True)

file_handler = logging.FileHandler(f"../notebooks_logging/06_ask_q4pages/{now_str}.log")
stream_handler = logging.StreamHandler()
# stdout_formatter = logging.Formatter(fmt="%(message)s") 
# stream_handler.setFormatter(stdout_formatter)
logging.basicConfig(
    encoding='utf-8', 
    format='%(asctime)s | %(levelname)s | %(message)s', 
    level=logging.INFO, 
    datefmt='%Y-%m-%dT%H:%M:%S',
    handlers=[
        file_handler,
        stream_handler,
    ],
    force=True
)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
li = logger.info
lw = logger.warning
li("start")
lw("check warning")

2025-02-27T16:53:55 | INFO | start
2025-02-27T16:53:55 | WARNING | check warning


In [4]:
df_temp_answers04 = pd.read_parquet(f"../data_temp/04 temp_answers.parquet")
df_temp_answers04 = df_temp_answers04.rename(columns={"documents_to_look_through": "ref_sha1"})
df_temp_answers04

,q_text,q_kind,company_names,company_names_subset,ref_sha1
0,"For Ziff Davis, Inc., what was the value of Cl...",number,"[Ziff Davis, Inc.]","[Ziff Davis, Inc.]",[ecabab4934d4b80570c4bb3b8e35b7476694b3fb]
1,Did Liberty Broadband Corporation announce a s...,boolean,[Liberty Broadband Corporation],[Liberty Broadband Corporation],[446545ae548543d8744f8d885ff75face3424ba4]
2,What is the total number of employees let go b...,number,[Pintec Technology Holdings Limited],[Pintec Technology Holdings Limited],[9e794a58e511f6a6a9a13b201d652deff9f9f69a]
3,Which leadership positions changed at Westwate...,names,"[Westwater Resources, Inc.]","[Westwater Resources, Inc.]",[92d9de8e4db96e0b95a484afcd1c54c6beb62c03]
4,Did Brave Bison Group plc mention any mergers ...,boolean,[Brave Bison Group plc],[Brave Bison Group plc],[ddd10e4612006205c4b1ba050a11648071e6e429]
...,...,...,...,...,...
95,"According to the annual report, what is the Ca...",number,[James Halstead plc],[James Halstead plc],[71d137454a1524843e1f49b34603438510232919]
96,What was the value of End-of-year tech staff h...,number,[archTIS Limited],[archTIS Limited],[c06d5ad4b6408fec26675d30b37a6042c007095a]
97,"For Westwater Resources, Inc., what was the va...",number,"[Westwater Resources, Inc.]","[Westwater Resources, Inc.]",[92d9de8e4db96e0b95a484afcd1c54c6beb62c03]
98,Which leadership positions changed at Origin B...,names,"[Origin Bancorp, Inc.]","[Origin Bancorp, Inc.]",[3f36d4f26ada778d89cf5a7344be0b9e9a5223a3]


In [5]:
df_subset = pd.read_parquet(f"../data_temp/04 subset.parquet")
df_subset

,sha1,cur,company_name,major_industry,mentions_recent_mergers_and_acquisitions,has_leadership_changes,has_layoffs,has_executive_compensation,has_rnd_investment_numbers,has_new_product_launches,...,has_dividend_policy_changes,has_share_buyback_plans,has_capital_structure_changes,mentions_new_risk_factors,has_guidance_updates,has_regulatory_or_litigation_issues,has_strategic_restructuring,has_supply_chain_disruptions,has_esg_initiatives,is_in_questions
0,0279901b645e568591ad95dac2c2bf939ef0c00d,USD,ACRES Commercial Realty Corp.,Financial Services,False,False,False,True,False,False,...,False,True,False,True,True,False,False,False,True,True
1,0981826b4b43a88920f3e01c71ae73539bab84cc,USD,Aptevo Therapeutics Inc.,Healthcare,True,False,False,True,True,True,...,False,False,False,True,False,True,False,True,True,True
2,0a61a353b1ea9fd9b8f63b60239634ca3007d58f,USD,Downer EDI Limited,Transport & Logistics,True,True,True,True,False,False,...,True,True,True,True,True,True,True,False,True,True
3,0c0faea14d108e1617f2d6d2a7c1aae04eb88fe0,USD,Odyssey Gold Limited,Pharmaceuticals,True,True,False,True,True,False,...,False,False,False,True,False,False,False,False,False,False
4,0f111d244aee3d976684995a222fa177a64571c4,USD,NextNav Inc.,Technology,True,False,False,True,True,True,...,False,False,False,True,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,f6a1077a46cb36761604f8f4840698e22837f5f7,USD,Aeeris Ltd,Technology,False,True,False,True,False,True,...,False,False,True,True,False,False,False,False,False,False
96,f7324f3868005b3aae6aafd30441c9bb6c8823e8,USD,Jefferies Financial Group Inc.,Financial Services,True,False,True,True,False,False,...,True,True,True,True,False,True,True,False,True,False
97,f85f0ccbbc59f1423d52d44e5ab70e3a01164499,USD,Integra LifeSciences Holdings Corporation,Healthcare,True,False,False,True,True,True,...,False,False,False,True,False,True,True,True,True,False
98,f879b3a802ccd6e8e6ca0a07ed8464318b7c0724,USD,Elixir Energy Limited,Energy and Utilities,True,False,False,True,True,True,...,False,False,True,True,False,False,False,True,True,True


In [11]:
sem = Semaphore(200)

In [7]:
client = openai.AsyncOpenAI(
    # This is the default and can be omitted
    api_key=openai_token,
)

In [22]:
# for question_id, row in df_temp_answers04.iterrows():
for question_id, row in df_temp_answers04.iloc[79:,:].iterrows():
            
    li(f"S---------question#{question_id} started")
    li(f"{question_id=}\n{row=}")
    ref_sha1_i = row["ref_sha1"][0]

    pdf_pages = list(pathlib.Path(f"../data_temp/05_cut_pdfs/{ref_sha1_i}/").glob('*'))
    ref_sha1 = row["ref_sha1"]
    for ref_sha1_i in row["ref_sha1"]:
        li(f"S---------question#{question_id} sha1={ref_sha1_i} started")
        pdf_pages = list(pathlib.Path(f"../data_temp/05_cut_pdfs/{ref_sha1_i}/").glob('*'))
        pdf_pages = sorted(pdf_pages)
        li(f'pdf_pages=\n{"\n".join(str(x) for x in pdf_pages)}')
        q_text = row["q_text"]
        li(f"{q_text=}")
        q_kind = row["q_kind"]
        li(f"{q_kind=}")

        match q_kind:
            case "number":
                result_type = 'Union[float, Literal["N/A"]]'
                result_description = 'A float number that is the answer to the question. If the answer is not found then "N/A"'
            case "boolean":
                result_type = 'Union[bool, Literal["N/A"]]'
                result_description = 'A boolean that is the answer to the question. If the answer is not found then "N/A"'
            case "name":
                result_type = 'Union[str, Literal["N/A"]]'
                result_description = 'A string representing a name that is the answer to the question. If the answer is not found then "N/A"'
            case "names":
                result_type = 'Union[List[str], Literal["N/A"]]'
                result_description = 'A list of strings that represent names that are the answer to the question. If the answer is not found then "N/A"'
        str_ans_class = f"""
class Answer(BaseModel):
    result: {result_type} = Field(..., description='{result_description}')
    result_prob: float = Field(..., description="Probability that the result is the right one")
    result_found: bool = Field(..., description="Is the answer to the question present in the file?")

"""

        exec(str_ans_class)
        # li(str_ans_class)


        prompt = f"""Please answer the following question based on the file provided: 
{q_text}

Check that the data is really present in the file.
Check that the found data cannot be confused with other things.
If the question is about currency amount then provide the answer in the currency units and not in thousands or millions. Check which units of measurement are given in the file and convert these units of measurements to currency units.

---
Output format:
The output should start with @@@ and end with @@@. If the result is empty it should still correspond to the JSON schema and the result should have N/A in result.
The output is a JSON that corresponds to the following pydantic class:
{str_ans_class}
        """

        li(f"pdf_sha1={ref_sha1_i}")
        li(f"q_text={q_text}")
        li(f"q_kind={q_kind}")
        li(f"prompt=\n{prompt}")

        async def send_one_page(page_i: int, pdf_page: str, prompt: str) -> openai.types.beta.threads.Run:
            async with sem:
                try:
                    pdf_assistant = await client.beta.assistants.create(
                        model="gpt-4o",
                        # model="o1-mini",
                        description="An assistant to extract the contents of PDF files.",
                        tools=[{"type": "file_search"}],
                        name="PDF assistant",
                        timeout=60
                    )
                except (httpx.HTTPError, openai.OpenAIError, openai.APITimeoutError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d

                # li(f"Assistent created")

                # Create thread
                try:
                    thread = await client.beta.threads.create(timeout=60)
                except (httpx.HTTPError, openai.OpenAIError, openai.APITimeoutError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d

                # li(f"thread created")
                try:
                    file = await client.files.create(file=open(pdf_page, "rb"), purpose="assistants", timeout=60)
                except (httpx.HTTPError, openai.OpenAIError, openai.APITimeoutError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d
                # li(f"file created")

                # Create assistant
                try:
                    await client.beta.threads.messages.create(
                        thread_id=thread.id,
                        role="user",
                        attachments=[
                            Attachment(
                                file_id=file.id, tools=[AttachmentToolFileSearch(type="file_search")]
                            )
                        ],
                        content=prompt,
                        timeout=60
                    )
                except (httpx.HTTPError, openai.OpenAIError, openai.APITimeoutError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d
                # li(f"message created")

                # Run thread
                try:
                    run = await client.beta.threads.runs.create_and_poll(
                        thread_id=thread.id, assistant_id=pdf_assistant.id, timeout=60, poll_interval_ms = 5000
                    )
                except (httpx.HTTPError, openai.OpenAIError, openai.APITimeoutError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d
                try:
                    messages_cursor = await client.beta.threads.messages.list(thread_id=run.thread_id, timeout=60)
                    messages = [message async for message in messages_cursor]
                except (httpx.HTTPError, openai.OpenAIError, openai.APITimeoutError) as e:
                    logger.exception("Error")
                    ans_d = {
                        "result": "",
                        "result_error": True
                    }
                    return page_i, pdf_page, ans_d                

                # Output text
                ans = messages[0].content[0].text.value

                ans_l_splitted = ans.split('@@@')
                if len(ans_l_splitted) == 3:
                    ans_splitted = ans_l_splitted[1]
                    try:
                        ans_d = json.loads(ans_splitted)
                        _ = Answer(**ans_d)
                    except:
                        ans_d = {
                            "result": ans_splitted,
                            "result_error": True
                        }
                else:
                    ans_d = {
                        "result": ans,
                        "result_error": True
                    }
            # if random.uniform(0, 1) < 0.1:
            #     ans_d = {
            #         "result": ans,
            #         "result_error": True
            #     }
            # return question_i, question, ans_d
                return page_i, pdf_page, ans_d
            
        tasks_done = ["0"]*len(pdf_pages)
        tasks_tries = {}
        tasks = set()
        li(f"Sending for one question started")
        for i, pdf_page in enumerate(pdf_pages):
            tasks_tries[i] = 1
            tasks.add(asyncio.create_task(send_one_page(i, pdf_page, prompt)))

        tasks_run = []
        while tasks:
            done, tasks = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
            for completed in done:
                i, param1, ans_d = await completed
                if "result_error" in ans_d:
                    
                    if tasks_tries[i] < 20:
                        if tasks_tries[i] > 5:
                            await asyncio.sleep(1)
                        lw(f"Task {i} failed. Total retries: {tasks_tries[i]}. Retrying")
                        tasks_tries[i] = tasks_tries[i] + 1
                        tasks.add(asyncio.create_task(send_one_page(i, param1, prompt)))
                    else:
                        lw(f"Task {i} failed. Total retries: {tasks_tries[i]}. Stop retrying. Returning the current answer with error.")
                        tasks_done[i] = "2"
                        tasks_run.append((i, param1, ans_d))
                        li("".join(tasks_done))
                else:
                    tasks_done[i] = "1"
                    tasks_run.append((i, param1, ans_d))
                    li("".join(tasks_done))

        tasks_run_sorted = sorted(tasks_run, key=lambda tup: tup[0])
        temp_answers = [ans_d for _, _, ans_d in tasks_run_sorted]
        df_temp_answers = pd.DataFrame(temp_answers)
        li("\n" + str(df_temp_answers))

        answer = {
            "question_id": question_id,
            "q_text": q_text,
            "ref_sha1": list(ref_sha1),
            "ref_sha1_i": ref_sha1_i,
            "ref_sha1_i_page": -1,
            "value": "N/A"
        }

        df_answers_filtered = df_temp_answers[df_temp_answers["result_found"] == True]
        if len(df_answers_filtered)>0:
            if q_kind == "names":
                df_temp_answers["result_str"] = df_temp_answers["result"].astype(str)
                df_answers_filtered = df_temp_answers[df_temp_answers["result_found"] == True]
                v_c_dict = df_answers_filtered["result_str"].value_counts().sort_values(ascending=False).to_dict()
            else:
                v_c_dict = df_answers_filtered["result"].value_counts().sort_values(ascending=False).to_dict()
            value_count_max = next(iter(v_c_dict.items()))
            if value_count_max[1] > 1:
                if q_kind == "names":
                    answer["value"] = df_answers_filtered[df_answers_filtered["result_str"]==value_count_max[0]]["result"].to_list()[0]
                    filter = df_answers_filtered["result_str"]==value_count_max[0]
                else:
                    answer["value"] = value_count_max[0]
                    filter = df_answers_filtered["result"] == value_count_max[0]
                answer["ref_sha1_i_page"] = df_answers_filtered[filter].index.to_list()[0]
            else:
                if q_kind == "number":
                    if len(df_answers_filtered["result"]) % 2 == 0:
                        res_min = df_answers_filtered["result"].min()
                        answer["value"] = df_answers_filtered[df_answers_filtered["result"]!=res_min]["result"].median()
                    else:
                        answer["value"] = df_answers_filtered["result"].median()
                    answer["ref_sha1_i_page"]= df_answers_filtered[df_answers_filtered["result"] == answer["value"]].index.to_list()[0]
                else:
                    answer["value"] = df_answers_filtered["result"].iloc[0]
                    answer["ref_sha1_i_page"] = df_answers_filtered.index.to_list()[0]
        li("\n" + str(df_answers_filtered))
        answer_json_str = json.dumps(answer, indent=4)
        li(answer_json_str)

        filename = f"q{question_id:04d}_{ref_sha1_i}.json"
        folder = pathlib.Path(f"../data_temp/06_ask_questions/")
        folder.mkdir(parents=True, exist_ok=True)
        full_filename = folder/ filename
        with open(full_filename, "w") as f:
            f.write(answer_json_str)

        filename_csv = f"q{question_id:04d}_{ref_sha1_i}.csv"
        filename_parquet = f"q{question_id:04d}_{ref_sha1_i}.parquet"
        folder = pathlib.Path(f"../data_temp/06_ask_questions_df_answers/")
        folder.mkdir(parents=True, exist_ok=True)
        full_filename_csv = folder/ filename_csv
        full_filename_parquet = folder/ filename_parquet
        
        df_answers_filtered.reset_index().to_csv(full_filename_csv, index=False)
        df_answers_filtered_list2str = df_answers_filtered.reset_index().copy()
        if q_kind == "names":
            df_answers_filtered_list2str["result"] = df_answers_filtered_list2str["result"].astype(str)
        df_answers_filtered_list2str.to_parquet(full_filename_parquet, index=False)

        li(f"----------question#{question_id} sha1={ref_sha1_i} finished")
    li(f"----------question#{question_id} finished")

2025-02-27T20:16:49 | INFO | S---------question#79 started
2025-02-27T20:16:49 | INFO | question_id=79
row=q_text                  Did SIG plc mention any mergers or acquisition...
q_kind                                                            boolean
company_names                                                   [SIG plc]
company_names_subset                                            [SIG plc]
ref_sha1                       [2db41ba86cc015db1f5f7de00b764a06e6de3dcc]
Name: 79, dtype: object
2025-02-27T20:16:49 | INFO | S---------question#79 sha1=2db41ba86cc015db1f5f7de00b764a06e6de3dcc started
2025-02-27T20:16:49 | INFO | pdf_pages=
..\data_temp\05_cut_pdfs\2db41ba86cc015db1f5f7de00b764a06e6de3dcc\2db41ba86cc015db1f5f7de00b764a06e6de3dcc_0000.pdf
..\data_temp\05_cut_pdfs\2db41ba86cc015db1f5f7de00b764a06e6de3dcc\2db41ba86cc015db1f5f7de00b764a06e6de3dcc_0001.pdf
..\data_temp\05_cut_pdfs\2db41ba86cc015db1f5f7de00b764a06e6de3dcc\2db41ba86cc015db1f5f7de00b764a06e6de3dcc_0002.pdf
..\dat